In [18]:
# imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
# perform camera calibration
images = glob.glob('camera_cal/calibration*.jpg')
object_pts = []
image_pts = []
nx = 9
ny = 6
objpts = np.zeros((ny*nx,3),np.float32)
objpts[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
for image in images:
    gray = cv2.cvtColor(plt.imread(image),cv2.COLOR_RGB2GRAY)
    ret, corners = cv2.findChessboardCorners(gray,(nx,ny),None)
    if ret == True:
        image_pts.append(corners)
        object_pts.append(objpts)
any = plt.imread(images[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_pts, image_pts, (nx,ny), None, None)

In [41]:
# undistort image
def undistort(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [5]:
# Use color transforms, gradients, etc., to create a thresholded binary image. 

In [6]:
# apply perspective transformationm  to above binary image

In [7]:
# detect lane pixels 

In [8]:
# fit the lane boundary

In [9]:
# Determine the curvature of the lane and vehicle position with respect to center

In [10]:
# Warp the detected lane boundaries back onto the original image

In [ ]:
# Output visual display of the lane boundaries and numerical estimation of lane curvature 
# and vehicle position